In [20]:
# Create an instance of the BettingApi
betting_api = cfbd.BettingApi(cfbd.ApiClient(configuration))

try:
    # Get betting lines for the specified year and week
    lines = betting_api.get_lines(year=2024, week=4)
    
    # Extract relevant betting data for Oklahoma
    betting_data = []
    for game in lines:
        if game.home_team == 'Oklahoma' or game.away_team == 'Oklahoma':
            for line in game.lines:
                betting_data.append({
                    'home_team': game.home_team,
                    'away_team': game.away_team,
                    'provider': line.provider,
                    'spread': line.spread,
                    'formatted_spread': line.formatted_spread,
                    'spread_open': line.spread_open,
                    'over_under': line.over_under,
                    'over_under_open': line.over_under_open,
                    'home_moneyline': line.home_moneyline,
                    'away_moneyline': line.away_moneyline,
                    'game_date': game.start_date
                })

    # Create a DataFrame to display the betting data
    betting_df = pd.DataFrame(betting_data)
    
    # Display the betting DataFrame
    print(betting_df)

except ApiException as e:
    print(f"Exception when calling BettingApi->get_lines: {e}")

  home_team  away_team    provider  spread formatted_spread  spread_open  \
0  Oklahoma  Tennessee    ESPN Bet     5.5   Tennessee -5.5          7.5   
1  Oklahoma  Tennessee  DraftKings     6.0     Tennessee -6         -3.0   
2  Oklahoma  Tennessee      Bovada     6.5   Tennessee -6.5         -3.5   

   over_under  over_under_open  home_moneyline  away_moneyline  \
0        56.5             57.5           190.0          -225.0   
1        57.0              NaN           185.0          -225.0   
2        57.0             58.0           215.0          -260.0   

                  game_date  
0  2024-09-21T23:30:00.000Z  
1  2024-09-21T23:30:00.000Z  
2  2024-09-21T23:30:00.000Z  


# Improved Code

In [31]:
import cfbd
import pandas as pd
from cfbd.rest import ApiException
from datetime import datetime, timedelta

# Configuration setup
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'aiS0bBk23RvX4eOWirh4JQgCK+UIO0k1AF/9A2RDyu2D6D/u1LLVKvdDKdHQokqF'
configuration.api_key_prefix['Authorization'] = 'Bearer'

# API instances
games_api = cfbd.GamesApi(cfbd.ApiClient(configuration))
stats_api = cfbd.StatsApi(cfbd.ApiClient(configuration))
betting_api = cfbd.BettingApi(cfbd.ApiClient(configuration))

def get_current_week():
    today = datetime.now()
    season_start = datetime(today.year, 8, 1)  # Assuming season starts in August
    week = ((today - season_start).days // 7) + 1
    return min(week, 15)  # Cap at week 15

def get_week_data(year, week, team):
    try:
        games = games_api.get_games(year=year, week=week, team=team)
        if not games:
            print(f"No games found for {team} in week {week} of {year}")
            return None, None, None, None
        
        game = games[0]
        print(f"Game data retrieved: {game}")
        opponent = game.away_team if game.home_team == team else game.home_team
        
        advanced_stats = stats_api.get_advanced_team_game_stats(year=year, week=week, team=team)
        if advanced_stats:
            print(f"Advanced stats retrieved: {advanced_stats[0]}")
        else:
            print("No advanced stats retrieved")
        
        return game, advanced_stats[0] if advanced_stats else None, opponent, game.id
    except ApiException as e:
        print(f"Exception when calling API: {e}")
        return None, None, None, None

def analyze_game(game, advanced_stats, team):
    analysis = {
        'home_team': game.home_team,
        'away_team': game.away_team,
        'date': game.start_date,
        'completed': game.completed
    }
    
    if game.completed:
        team_score = game.home_points if game.home_team == team else game.away_points
        opponent_score = game.away_points if game.home_team == team else game.home_points
        
        if team_score is not None and opponent_score is not None:
            analysis['result'] = 'won' if team_score > opponent_score else 'lost'
            analysis['score'] = f"{team_score}-{opponent_score}"
        else:
            analysis['result'] = 'unknown'
            analysis['score'] = 'N/A'
        
        if advanced_stats:
            offense = advanced_stats.offense
            defense = advanced_stats.defense
            
            analysis.update({
                'plays': getattr(offense, 'plays', 'N/A'),
                'yards_per_play': round(getattr(offense, 'yards_per_play', 0), 2),
                'success_rate': f"{getattr(offense, 'success_rate', 0) * 100:.1f}%",
                'explosiveness': round(getattr(offense, 'explosiveness', 0), 2),
                'opponent_plays': getattr(defense, 'plays', 'N/A'),
                'opponent_yards_per_play': round(getattr(defense, 'yards_per_play', 0), 2),
                'opponent_success_rate': f"{getattr(defense, 'success_rate', 0) * 100:.1f}%",
                'opponent_explosiveness': round(getattr(defense, 'explosiveness', 0), 2),
            })
    else:
        analysis['result'] = 'upcoming'
        analysis['score'] = 'Not played yet'
    
    return analysis

def get_betting_data(year, week, team):
    try:
        lines = betting_api.get_lines(year=year, week=week)
        betting_data = [
            {
                'home_team': game.home_team,
                'away_team': game.away_team,
                'provider': line.provider,
                'spread': line.spread,
                'formatted_spread': line.formatted_spread,
                'over_under': line.over_under,
                'home_moneyline': line.home_moneyline,
                'away_moneyline': line.away_moneyline,
                'game_date': game.start_date
            }
            for game in lines
            for line in game.lines
            if game.home_team == team or game.away_team == team
        ]
        return pd.DataFrame(betting_data)
    except ApiException as e:
        print(f"Exception when calling BettingApi->get_lines: {e}")
        return pd.DataFrame()

def generate_insights(game_analysis, betting_df, team):
    insights = []
    
    if game_analysis['completed']:
        insights.extend([
            f"{'Great win' if game_analysis['result'] == 'won' else 'Tough loss'} against {game_analysis['away_team'] if game_analysis['home_team'] == team else game_analysis['home_team']}",
            f"Score: {game_analysis['score']}",
        ])
        
        if 'plays' in game_analysis:
            insights.extend([
                f"Ran {game_analysis['plays']} plays vs opponent's {game_analysis['opponent_plays']}",
                f"Averaged {game_analysis['yards_per_play']} yards per play vs opponent's {game_analysis['opponent_yards_per_play']}",
                f"Offensive success rate: {game_analysis['success_rate']}",
                f"Offensive explosiveness: {game_analysis['explosiveness']}",
            ])
    else:
        insights.extend([
            f"Upcoming game against {game_analysis['away_team'] if game_analysis['home_team'] == team else game_analysis['home_team']}",
            f"Date: {game_analysis['date']}",
        ])
    
    if not betting_df.empty:
        avg_spread = betting_df['spread'].mean()
        avg_over_under = betting_df['over_under'].mean()
        insights.extend([
            f"\nBetting Insights:",
            f"Average spread: {avg_spread:.1f}",
            f"Average over/under: {avg_over_under:.1f}",
            f"{team} is the {'home' if betting_df.iloc[0]['home_team'] == team else 'away'} team",
            f"{'Favored' if (betting_df.iloc[0]['home_team'] == team) == (avg_spread < 0) else 'Underdog'} by {abs(avg_spread):.1f} points"
        ])
    
    return "\n".join(insights)

def main():
    team = 'Oklahoma'
    year = datetime.now().year
    current_week = get_current_week()
    
    # Analyze current week's game
    current_game, current_stats, opponent, _ = get_week_data(year, current_week, team)
    if current_game:
        game_analysis = analyze_game(current_game, current_stats, team)
        
        # Get betting data for the current/upcoming game
        betting_df = get_betting_data(year, current_week, team)
        
        # Generate insights
        insights = generate_insights(game_analysis, betting_df, team)
        
        print("\nGenerated Insights:")
        print(insights)
        
        if not betting_df.empty:
            print("\nDetailed Betting Data:")
            print(betting_df)
    else:
        print(f"No game data found for {team} in week {current_week} of {year}")

if __name__ == "__main__":
    main()

Game data retrieved: {'attendance': None,
 'away_conference': 'SEC',
 'away_division': 'fbs',
 'away_id': 201,
 'away_line_scores': None,
 'away_points': None,
 'away_post_win_prob': None,
 'away_postgame_elo': None,
 'away_pregame_elo': 1829,
 'away_team': 'Oklahoma',
 'completed': False,
 'conference_game': True,
 'excitement_index': None,
 'highlights': None,
 'home_conference': 'SEC',
 'home_division': 'fbs',
 'home_id': 145,
 'home_line_scores': None,
 'home_points': None,
 'home_post_win_prob': None,
 'home_postgame_elo': None,
 'home_pregame_elo': 1900,
 'home_team': 'Ole Miss',
 'id': 401628402,
 'neutral_site': False,
 'notes': 'EARLY: Noon - 1pm ET start',
 'season': 2024,
 'season_type': 'regular',
 'start_date': '2024-10-26T04:00:00.000Z',
 'start_time_tbd': True,
 'venue': 'Vaught-Hemingway Stadium',
 'venue_id': 3974,
 'week': 9}
No advanced stats retrieved

Generated Insights:
Upcoming game against Ole Miss
Date: 2024-10-26T04:00:00.000Z

Betting Insights:
Average spread

In [33]:
import cfbd
import pandas as pd
from cfbd.rest import ApiException
from datetime import datetime, timedelta

# Configuration setup
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'aiS0bBk23RvX4eOWirh4JQgCK+UIO0k1AF/9A2RDyu2D6D/u1LLVKvdDKdHQokqF'
configuration.api_key_prefix['Authorization'] = 'Bearer'

# API instances
games_api = cfbd.GamesApi(cfbd.ApiClient(configuration))
stats_api = cfbd.StatsApi(cfbd.ApiClient(configuration))
betting_api = cfbd.BettingApi(cfbd.ApiClient(configuration))

def get_current_week():
    today = datetime.now()
    season_start = datetime(today.year, 8, 1)  # Assuming season starts in August
    week = ((today - season_start).days // 7) + 1
    return min(week, 15)  # Cap at week 15

def get_week_data(year, week, team):
    try:
        games = games_api.get_games(year=year, week=week, team=team)
        if not games:
            print(f"No games found for {team} in week {week} of {year}")
            return None, None, None, None
        
        game = games[0]
        print(f"Game data retrieved: {game}")
        opponent = game.away_team if game.home_team == team else game.home_team
        
        advanced_stats = stats_api.get_advanced_team_game_stats(year=year, week=week, team=team)
        if advanced_stats:
            print(f"Advanced stats retrieved: {advanced_stats[0]}")
        else:
            print("No advanced stats retrieved")
        
        return game, advanced_stats[0] if advanced_stats else None, opponent, game.id
    except ApiException as e:
        print(f"Exception when calling API: {e}")
        return None, None, None, None

def analyze_game(game, advanced_stats, team):
    analysis = {
        'home_team': game.home_team,
        'away_team': game.away_team,
        'date': game.start_date,
        'completed': game.completed,
        'venue': game.venue,
        'notes': game.notes
    }
    
    if game.completed:
        team_score = game.home_points if game.home_team == team else game.away_points
        opponent_score = game.away_points if game.home_team == team else game.home_points
        
        if team_score is not None and opponent_score is not None:
            analysis['result'] = 'won' if team_score > opponent_score else 'lost'
            analysis['score'] = f"{team_score}-{opponent_score}"
        else:
            analysis['result'] = 'unknown'
            analysis['score'] = 'N/A'
        
        if advanced_stats:
            offense = advanced_stats.offense
            defense = advanced_stats.defense
            
            analysis.update({
                'plays': getattr(offense, 'plays', 'N/A'),
                'yards_per_play': round(getattr(offense, 'yards_per_play', 0), 2),
                'success_rate': f"{getattr(offense, 'success_rate', 0) * 100:.1f}%",
                'explosiveness': round(getattr(offense, 'explosiveness', 0), 2),
                'opponent_plays': getattr(defense, 'plays', 'N/A'),
                'opponent_yards_per_play': round(getattr(defense, 'yards_per_play', 0), 2),
                'opponent_success_rate': f"{getattr(defense, 'success_rate', 0) * 100:.1f}%",
                'opponent_explosiveness': round(getattr(defense, 'explosiveness', 0), 2),
            })
    else:
        analysis['result'] = 'upcoming'
        analysis['score'] = 'Not played yet'
    
    return analysis

def get_betting_data(year, week, team):
    try:
        lines = betting_api.get_lines(year=year, week=week)
        betting_data = [
            {
                'home_team': game.home_team,
                'away_team': game.away_team,
                'provider': line.provider,
                'spread': line.spread,
                'formatted_spread': line.formatted_spread,
                'over_under': line.over_under,
                'home_moneyline': line.home_moneyline,
                'away_moneyline': line.away_moneyline,
                'game_date': game.start_date
            }
            for game in lines
            for line in game.lines
            if game.home_team == team or game.away_team == team
        ]
        return pd.DataFrame(betting_data)
    except ApiException as e:
        print(f"Exception when calling BettingApi->get_lines: {e}")
        return pd.DataFrame()

def format_date(date_string):
    date = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S.%fZ")
    et_time = date - timedelta(hours=4)  # Convert to Eastern Time (approximation)
    return et_time.strftime("%A, %B %d, %Y at %I:%M %p ET")

def generate_insights(game_analysis, betting_df, team):
    insights = []
    
    opponent = game_analysis['away_team'] if game_analysis['home_team'] == team else game_analysis['home_team']
    
    if game_analysis['completed']:
        insights.extend([
            f"{'Great win' if game_analysis['result'] == 'won' else 'Tough loss'} against {opponent}",
            f"Score: {game_analysis['score']}",
        ])
        
        if 'plays' in game_analysis:
            insights.extend([
                f"Ran {game_analysis['plays']} plays vs opponent's {game_analysis['opponent_plays']}",
                f"Averaged {game_analysis['yards_per_play']} yards per play vs opponent's {game_analysis['opponent_yards_per_play']}",
                f"Offensive success rate: {game_analysis['success_rate']}",
                f"Offensive explosiveness: {game_analysis['explosiveness']}",
            ])
    else:
        insights.extend([
            f"Upcoming game: {team} vs {opponent}",
            f"Date: {format_date(game_analysis['date'])}",
            f"Venue: {game_analysis['venue']}",
        ])
        if game_analysis['notes']:
            insights.append(f"Notes: {game_analysis['notes']}")
    
    if not betting_df.empty:
        avg_spread = betting_df['spread'].mean()
        avg_over_under = betting_df['over_under'].mean()
        
        insights.append("\nBetting Insights:")
        if not pd.isna(avg_spread):
            insights.append(f"Average spread: {avg_spread:.1f}")
        else:
            insights.append("Spread: Not available")
        
        if not pd.isna(avg_over_under):
            insights.append(f"Average over/under: {avg_over_under:.1f}")
        else:
            insights.append("Over/under: Not available")
        
        is_home = betting_df.iloc[0]['home_team'] == team
        insights.append(f"{team} is the {'home' if is_home else 'away'} team")
        
        if not pd.isna(avg_spread):
            favored = (is_home and avg_spread < 0) or (not is_home and avg_spread > 0)
            insights.append(f"{team} is the {'favorite' if favored else 'underdog'} by {abs(avg_spread):.1f} points")
        
        if not pd.isna(betting_df.iloc[0]['home_moneyline']) and not pd.isna(betting_df.iloc[0]['away_moneyline']):
            team_moneyline = betting_df.iloc[0]['home_moneyline'] if is_home else betting_df.iloc[0]['away_moneyline']
            insights.append(f"{team}'s moneyline: {team_moneyline}")
    else:
        insights.append("\nNo betting data available for this game yet.")
    
    return "\n".join(insights)


Analyzing Oklahoma's game for Week 9 of 2024
Game data retrieved: {'attendance': None,
 'away_conference': 'SEC',
 'away_division': 'fbs',
 'away_id': 201,
 'away_line_scores': None,
 'away_points': None,
 'away_post_win_prob': None,
 'away_postgame_elo': None,
 'away_pregame_elo': 1829,
 'away_team': 'Oklahoma',
 'completed': False,
 'conference_game': True,
 'excitement_index': None,
 'highlights': None,
 'home_conference': 'SEC',
 'home_division': 'fbs',
 'home_id': 145,
 'home_line_scores': None,
 'home_points': None,
 'home_post_win_prob': None,
 'home_postgame_elo': None,
 'home_pregame_elo': 1900,
 'home_team': 'Ole Miss',
 'id': 401628402,
 'neutral_site': False,
 'notes': 'EARLY: Noon - 1pm ET start',
 'season': 2024,
 'season_type': 'regular',
 'start_date': '2024-10-26T04:00:00.000Z',
 'start_time_tbd': True,
 'venue': 'Vaught-Hemingway Stadium',
 'venue_id': 3974,
 'week': 9}
No advanced stats retrieved

Generated Insights:
Upcoming game: Oklahoma vs Ole Miss
Date: Saturday

In [35]:
def get_advanced_box_score(game_id):
    try:
        box_score = games_api.get_advanced_box_score(game_id)
        return box_score
    except ApiException as e:
        print(f"Exception when calling GamesApi->get_advanced_box_score: {e}")
        return None

def analyze_advanced_box_score(box_score, team):
    if not box_score or not box_score.teams:
        return "Advanced box score data not available."

    analysis = []
    team_data = next((t for t in box_score.teams.ppa if t.team == team), None)
    if not team_data:
        return "Team data not found in advanced box score."

    # PPA analysis
    ppa_overall = team_data.overall
    analysis.append("Predicted Points Added (PPA) by quarter:")
    analysis.append(f"Q1: {ppa_overall.quarter1:.2f}, Q2: {ppa_overall.quarter2:.2f}, Q3: {ppa_overall.quarter3:.2f}, Q4: {ppa_overall.quarter4:.2f}")
    analysis.append(f"Total PPA: {ppa_overall.total:.2f}")

    # Success Rates
    success_rates = next((sr for sr in box_score.teams.success_rates if sr.team == team), None)
    if success_rates:
        overall_sr = success_rates.overall
        analysis.append("\nSuccess Rates by quarter:")
        analysis.append(f"Q1: {overall_sr.quarter1:.2%}, Q2: {overall_sr.quarter2:.2%}, Q3: {overall_sr.quarter3:.2%}, Q4: {overall_sr.quarter4:.2%}")
        analysis.append(f"Overall Success Rate: {overall_sr.total:.2%}")

    # Explosiveness
    explosiveness = next((exp for exp in box_score.teams.explosiveness if exp.team == team), None)
    if explosiveness:
        overall_exp = explosiveness.overall
        analysis.append("\nExplosiveness by quarter:")
        analysis.append(f"Q1: {overall_exp.quarter1:.2f}, Q2: {overall_exp.quarter2:.2f}, Q3: {overall_exp.quarter3:.2f}, Q4: {overall_exp.quarter4:.2f}")
        analysis.append(f"Overall Explosiveness: {overall_exp.total:.2f}")

    return "\n".join(analysis)

def generate_insights(game_analysis, betting_df, team, advanced_analysis):
    insights = []
    
    opponent = game_analysis['away_team'] if game_analysis['home_team'] == team else game_analysis['home_team']
    
    if game_analysis['completed']:
        insights.extend([
            f"{'Great win' if game_analysis['result'] == 'won' else 'Tough loss'} against {opponent}",
            f"Score: {game_analysis['score']}",
            "\nAdvanced Analysis:",
            advanced_analysis
        ])
        
        if 'plays' in game_analysis:
            insights.extend([
                f"\nRan {game_analysis['plays']} plays vs opponent's {game_analysis['opponent_plays']}",
                f"Averaged {game_analysis['yards_per_play']} yards per play vs opponent's {game_analysis['opponent_yards_per_play']}",
                f"Offensive success rate: {game_analysis['success_rate']}",
                f"Offensive explosiveness: {game_analysis['explosiveness']}",
            ])
    else:
        insights.extend([
            f"Upcoming game: {team} vs {opponent}",
            f"Date: {format_date(game_analysis['date'])}",
            f"Venue: {game_analysis['venue']}",
        ])
        if game_analysis['notes']:
            insights.append(f"Notes: {game_analysis['notes']}")
    
    # ... (keep the existing betting insights code)
    
    return "\n".join(insights)



In [36]:
def main():
    team = 'Oklahoma'
    year = datetime.now().year
    current_week = get_current_week()
    
    print(f"Analyzing {team}'s game for Week {current_week} of {year}")
    
    try:
        # Analyze current week's game
        print("Fetching game data...")
        current_game, current_stats, opponent, game_id = get_week_data(year, current_week, team)
        
        if current_game:
            print(f"Game data retrieved for {team} vs {opponent}")
            game_analysis = analyze_game(current_game, current_stats, team)
            print("Game analysis completed")
            
            # Get advanced box score data
            print("Fetching advanced box score data...")
            advanced_box_score = get_advanced_box_score(game_id)
            if advanced_box_score:
                print("Advanced box score data retrieved")
                advanced_analysis = analyze_advanced_box_score(advanced_box_score, team)
                print("Advanced analysis completed")
            else:
                print("No advanced box score data available")
                advanced_analysis = "No advanced data available"
            
            # Get betting data for the current/upcoming game
            print("Fetching betting data...")
            betting_df = get_betting_data(year, current_week, team)
            if not betting_df.empty:
                print("Betting data retrieved")
            else:
                print("No betting data available")
            
            # Generate insights
            print("Generating insights...")
            insights = generate_insights(game_analysis, betting_df, team, advanced_analysis)
            
            print("\nGenerated Insights:")
            print(insights)
            
            if not betting_df.empty:
                print("\nDetailed Betting Data:")
                print(betting_df.to_string(index=False))
        else:
            print(f"No game data found for {team} in week {current_week} of {year}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Analyzing Oklahoma's game for Week 9 of 2024
Fetching game data...
Game data retrieved: {'attendance': None,
 'away_conference': 'SEC',
 'away_division': 'fbs',
 'away_id': 201,
 'away_line_scores': None,
 'away_points': None,
 'away_post_win_prob': None,
 'away_postgame_elo': None,
 'away_pregame_elo': 1829,
 'away_team': 'Oklahoma',
 'completed': False,
 'conference_game': True,
 'excitement_index': None,
 'highlights': None,
 'home_conference': 'SEC',
 'home_division': 'fbs',
 'home_id': 145,
 'home_line_scores': None,
 'home_points': None,
 'home_post_win_prob': None,
 'home_postgame_elo': None,
 'home_pregame_elo': 1900,
 'home_team': 'Ole Miss',
 'id': 401628402,
 'neutral_site': False,
 'notes': 'EARLY: Noon - 1pm ET start',
 'season': 2024,
 'season_type': 'regular',
 'start_date': '2024-10-26T04:00:00.000Z',
 'start_time_tbd': True,
 'venue': 'Vaught-Hemingway Stadium',
 'venue_id': 3974,
 'week': 9}
No advanced stats retrieved
Game data retrieved for Oklahoma vs Ole Miss
Game